<a href="https://colab.research.google.com/github/anirbanghoshsbi/.github.io/blob/master/Get_return_for_ml_stock_selection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [9]:
import pandas as pd
import numpy as np

# Load the data
df = pd.read_csv('https://raw.githubusercontent.com/anirbanghoshsbi/data/main/data_nifty50.csv')

# Convert the date column to datetime
df['date'] = pd.to_datetime(df['date'])

# Sort the dataframe by ticker and date
df = df.sort_values(['ticker', 'date'])

# Group the dataframe by ticker
grouped = df.groupby('ticker')

# Function to calculate weekly returns
def calculate_weekly_return(group):
    # Shift the 'adj close' column by 1 day (1 month)
    group['next_week_close'] = group['adj close'].shift(-1)

    # Calculate the weekly return
    group['weekly_return'] = (group['next_week_close'] - group['adj close']) / group['adj close']

    return group

# Apply the function to each group (ticker)
df_with_returns = grouped.apply(calculate_weekly_return)

# Reset the index
df_with_returns = df_with_returns.reset_index(drop=True)

# Remove rows where we couldn't calculate the weekly return (last week of data for each stock)
df_with_returns = df_with_returns.dropna(subset=['weekly_return'])

# Display the first few rows of the result
print(df_with_returns[['date', 'ticker', 'adj close', 'next_week_close', 'weekly_return']].head())

# Calculate summary statistics of weekly returns for each stock
weekly_return_summary = df_with_returns.groupby('ticker')['weekly_return'].agg(['mean', 'median', 'std', 'min', 'max'])
print("\nWeekly Return Summary Statistics:")
print(weekly_return_summary)

# Find the stock with the highest average weekly return
best_performing_stock = weekly_return_summary['mean'].idxmax()
print(f"\nBest performing stock based on average weekly return: {best_performing_stock}")

# Save the results to a CSV file
df_with_returns.to_csv('stock_mothly_returns.csv', index=False)
print("\nResults saved to 'stock_weekly_returns.csv'")

        date       ticker  adj close  next_week_close  weekly_return
0 2016-11-30  ADANIENT.NS  36.299690        40.833820       0.124908
1 2016-12-31  ADANIENT.NS  40.833820        46.488136       0.138471
2 2017-01-31  ADANIENT.NS  46.488136        52.542538       0.130235
3 2017-02-28  ADANIENT.NS  52.542538        57.903469       0.102030
4 2017-03-31  ADANIENT.NS  57.903469        58.116852       0.003685

Weekly Return Summary Statistics:
                   mean    median       std       min       max
ticker                                                         
ADANIENT.NS    0.068998  0.062332  0.185218 -0.541393  0.732816
ADANIPORTS.NS  0.018327  0.018835  0.095570 -0.257518  0.326074
APOLLOHOSP.NS  0.023780  0.026196  0.108433 -0.343790  0.334510
ASIANPAINT.NS  0.017882  0.010163  0.069918 -0.129192  0.247912
AXISBANK.NS    0.014478  0.017324  0.093986 -0.456475  0.235679
BAJAJ-AUTO.NS  0.013052  0.013415  0.074106 -0.268800  0.297154
BAJAJFINSV.NS  0.028987  0.022156  0.13